In [1]:
import sys
sys.path.append('CONFIG_files/')

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import shutil

from matplotlib import pyplot as plt
from tabulate import tabulate

import os, shutil, time, pdb, random
import scipy.stats as stats 
import scipy

from math import pi
from datetime import datetime
from collections import OrderedDict
import pickle

import torch
from torch.utils.data import TensorDataset, DataLoader

from importlib import reload
from models import *
from utils import *
from runmanager import *
from experiment import *
from plot_utils import *
from preprocessing_utils import *
from analysis_seasonal import * 

from sklearn.metrics import mean_squared_error as mse

import matplotlib
matplotlib.rc_file_defaults()
%matplotlib inline

# import CONFIG_SUSHIWAT_BGMM_23_JUNE_2023 as C
import CONFIG as C

pd.options.display.max_columns = None

np.random.seed(4)

%load_ext autoreload
%autoreload 2

device = C.device

C.REGION, C.RUN_NAME, C.split_by

('COMBINED', 'COMBINED_BGMM_07_NOV_2023_EXP_1', 'station')

### Import data

In [3]:
data = DataPreprocessing(train_path=C.TRAIN_PATH, start=C.start, end=C.end, 
                add_yesterday = False, 
                basin_filter = None, 
                split_bias_corrected_only = False, 
                filter_incomplete_years = False, 
                include_non_bc_stations = True, 
                split_by = C.split_by)

### Split data into held out sets for K-fold cross validation

In [4]:
if C.SORT_BY_ELEVATION:
    data.split_stations(sort_by_elev = True)
else:
    data.split_stations(sort_by_elev = False)
    
print(C.SORT_BY_ELEVATION)

False


### Prepare data

In [5]:
if C.ADD_PREVIOUS_DAY:
    C.predictors.append('obs_yesterday')

data.input_data(C.predictors, C.predictand, sort_by_quantile=C.sort_by_quantile)

print(C.ADD_PREVIOUS_DAY)
print(C.sort_by_quantile)

False
False


## Multi-Run: Train model with different hyperparameters

### Model run

In [7]:
experiment_label = C.RUN_NAME

experiment_label, C.K_FOLD

('COMBINED_BGMM_09_NOV_2023_EXP_2', 10)

In [8]:
load_run = None

In [9]:
if load_run == None:
    src = 'CONFIG.py'
    dst = f'CONFIG_files/CONFIG_{experiment_label}.py'
    shutil.copy(src, dst)

st_test, predictions, importances = multirun(data, C.predictors, C.params, C.epochs, 
                                split_by = 'station', sequential_samples = True, n_samples=1, 
                                load_run = load_run, #None, 
                                experiment_label = experiment_label, feature_attribution = True, 
                                show_loss_plot=False)

,run,epoch,loss,valid_loss,test_loss,decision_loss,epoch duration,run duration,lr,batch_size,likelihood_fn,dropout_rate,random_noise,k,model_arch
0,1,1,1.283150,1.125735,1.251856,1.125735,14.903860,14.921769,0.001,32,bgmm,0,0,0,"(VGLM, [])"
1,1,2,1.205419,1.123854,1.251637,1.123854,14.213668,30.071979,0.001,32,bgmm,0,0,0,"(VGLM, [])"
2,1,3,1.203991,1.122947,1.252874,1.122947,14.051169,44.146604,0.001,32,bgmm,0,0,0,"(VGLM, [])"
3,1,4,1.203121,1.121859,1.249670,1.121859,14.323198,58.516319,0.001,32,bgmm,0,0,0,"(VGLM, [])"
4,1,5,1.202783,1.124523,1.248176,1.124523,15.106483,73.691222,0.001,32,bgmm,0,0,0,"(VGLM, [])"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,8,8,1.591516,1.528922,1.693231,1.528922,42.389566,333.761155,0.001,32,bgmm,0,0,1,"(SimpleRNN, [10])"
78,8,9,1.590839,1.526480,1.690071,1.526480,42.427782,376.270917,0.001,32,bgmm,0,0,1,"(SimpleRNN, [10])"
79,8,10,1.590116,1.524706,1.690315,1.524706,41.957183,418.320185,0.001,32,bgmm,0,0,1,"(SimpleRNN, [10])"
80,9,1,1.185395,1.074925,1.263685,1.074925,18.884199,20.089280,0.001,32,bgmm,0,0,1,"(MLP, [50, 50])"


KeyboardInterrupt: 

In [14]:
# table_of_predictions_ks_test(predictions, C.seasons, columns, sample_cols, add_cols)

In [17]:
# table_of_predictions_for_metric(predictions, C.seasons, columns, C.n_samples, sample_cols, add_cols, metric = 'smape', prefix='smape')

In [18]:
# table_of_predictions_for_metric(predictions, C.seasons, columns, C.n_samples, sample_cols, add_cols, metric = 'edd', prefix='edd')

In [19]:
# table_of_predictions_for_metric(predictions, C.seasons, columns, C.n_samples, sample_cols, add_cols, metric = 'ae', prefix='ae')

In [ ]:
a = pd.read_csv('results.csv')

b = a.groupby(['k','run']).agg({'valid_loss': 'min', 
                                 'model_arch': 'first', 
                                 'likelihood_fn': 'first',
                                 'lr':'first',
                                 'batch_size':'first',
                                 'dropout_rate':'first',
                               })

c = b.groupby(['run']).agg({'valid_loss': 'mean', 
                        'model_arch': 'first', 
                        'likelihood_fn': 'first',
                        'lr':'first',
                        'batch_size':'first',
                        'dropout_rate':'first',
                         }
                   ).sort_values('valid_loss').reset_index()

c.groupby(['model_arch',
           'likelihood_fn',
           'lr',
           'batch_size',
           'dropout_rate',
           ]).agg({'valid_loss': 'mean'}
                   ).sort_values('valid_loss').reset_index()